# Hands on Machine Learning (ML) for Materials

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paolodeangelis/AEM/blob/main/2_Hands_on_ML_for_Materials.ipynb)

# NN Model Regressor

## Introduction

In this hands-on tutorial, we will explore how to **predict the elastic properties** of crystalline materials using a machine learning approach. Specifically, we will show how to:

1. Load a dataset of structures and their associated elastic tensors.
2. Clean and preprocess the data (removing outliers, normalizing target values, etc.).
3. Represent each structure with a **SOAP** (Smooth Overlap of Atomic Positions) descriptor.
4. Train a simple **neural network** model in **PyTorch** to predict the diagonal components of the elastic tensor ($\mathrm{C}_{11}, \mathrm{C}_{22}, \mathrm{C}_{33}, \mathrm{C}_{44}, \mathrm{C}_{55}, \mathrm{C}_{66}$).
5. Evaluate model performance using metrics such as **mean absolute error** (MAE) and **$R^2$**.
6. [Optional] Compare these predictions with a direct **DFT-based approach** (via MACE) for calculating certain elastic constants.

## Theoretical Context

### Elastic Tensor

A second-order elastic stiffness tensor $\mathbf{C}$ in 3D has 36 components, which are typically reduced to 21 independent components by symmetry. It generalizes **Hooke's law** and **completely** describes the elastic **behavior** of the material.

$$
    \begin{pmatrix}
    \sigma_{11}\\[6pt]
    \sigma_{22}\\[6pt]
    \sigma_{33}\\[6pt]
    \sigma_{23}\\[6pt]
    \sigma_{13}\\[6pt]
    \sigma_{12}
    \end{pmatrix}
    =
    \underbrace{\begin{pmatrix}
    C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16}\\[6pt]
    C_{21} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26}\\[6pt]
    C_{31} & C_{32} & C_{33} & C_{34} & C_{35} & C_{36}\\[6pt]
    C_{41} & C_{42} & C_{43} & C_{44} & C_{45} & C_{46}\\[6pt]
    C_{51} & C_{52} & C_{53} & C_{54} & C_{55} & C_{56}\\[6pt]
    C_{61} & C_{62} & C_{63} & C_{64} & C_{65} & C_{66}
    \end{pmatrix}}_{\displaystyle \text{(6×6 stiffness matrix)}}
    \begin{pmatrix}
    \varepsilon_{11}\\[6pt]
    \varepsilon_{22}\\[6pt]
    \varepsilon_{33}\\[6pt]
    2\,\varepsilon_{23}\\[6pt]
    2\,\varepsilon_{13}\\[6pt]
    2\,\varepsilon_{12}
    \end{pmatrix}.
$$



Here we focus on the diagonal components:

$$
C_{11}, \quad C_{22}, \quad C_{33}, \quad C_{44}, \quad C_{55}, \quad C_{66}.
$$

In the most common engineering notation, the linear relation between stress $\sigma$ and strain $\varepsilon$ can be written as:

$$
\sigma_{ij} = \sum_{k, l} C_{ijkl} \,\varepsilon_{kl},
$$

where $\sigma_{ij}$ is the stress tensor, $\varepsilon_{kl}$ is the strain tensor, and $C_{ijkl}$ are the elastic stiffness constants.

## [Step 1]: Install library and download data

In [ ]:
%pip install torch torchvision pandas matplotlib numpy pymatgen dscribe scikit-learn tqdm mp-api pandas iterative-stratification mace-torch nglview 

In [ ]:
import numpy as np
import pandas as pd
import torch
from ase import Atoms
from dscribe.descriptors import SOAP
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from pymatgen.core import Composition, Structure
from scipy.stats import zscore
from torch.utils.data import DataLoader, Dataset

> **Note**: The following setting must be changed in the assignment.

In [ ]:
N_SAMPLES = 5000  # Number of samples to use
SEED = 42  # Random seed to ensure reproducibility
BATCH_SIZE = 8  # Batch size

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.set_default_dtype(torch.float32)

dowload data

In [ ]:
!wget https://github.com/paolodeangelis/AEM/raw/refs/heads/main/data/elasticity_full_properties.h5

Load data:

In [ ]:
df_full = pd.read_hdf("elasticity_full_properties.h5", key="df")

df_full

## [Step 2]: Data cleaning & Downsampling

**Data Cleaning Steps**:

1. **Drop missing data** in `"Structure"` or `"Elastic Tensor"`.  
2. **Extract diagonal components** $(C_{11}, C_{22}, \ldots, C_{66})$ from the elastic tensor.  
3. **Remove outliers** using the IQR method, filtering extreme values in each diagonal component.  
4. **Shuffle** the remaining dataset randomly.  
5. **Select** only the first `N_SAMPLES` rows for practical training.


In [ ]:
# Drop missing data
df_full = df_full.dropna(subset=["Structure", "Elastic Tensor"])

In [ ]:
# Extract targets
tensor_components = ["C11", "C22", "C33", "C44", "C55", "C66"]
tensor_indices = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]

for i, (r, c) in enumerate(tensor_indices):
    df_full[tensor_components[i]] = df_full["Elastic Tensor"].apply(
        lambda x: np.array(x)[r, c]
    )

In [ ]:
# IQR-based outlier removal for each tensor component


def remove_outliers_iqr(df, cols):
    df_clean = df.copy()
    for col in cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[
            (df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)
        ]
    return df_clean.reset_index(drop=True)

In [ ]:
df_clean = remove_outliers_iqr(df_full, tensor_components)
print(f"Removed {len(df_full)-len(df_clean)} outliers. Remaining data: {len(df_clean)}")

Downsampling

In [ ]:
df_clean_shuffled = df_clean.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Select only first N_SAMPLES
df_final = df_clean_shuffled.iloc[:N_SAMPLES].reset_index(drop=True)
print(f"Final dataset size: {len(df_final)}")

## [Step 3]: Data Splitting 

Instead of a purely random split, we perform a **multilabel stratified split** based on the presence or absence of each chemical element in the structures. Here's why:

1. **Element Distribution**: A simple random split may result in some elements being over-represented in either the training or test set (or missing entirely in one of them).  
2. **Balanced Training**: By using a stratified approach, we ensure that each element’s frequency distribution is maintained across both sets, giving the model a more representative range of materials during training and an unbiased evaluation on the test set.  
3. **Improved Robustness**: Preserving distribution across all labels (elements) reduces the risk of skewing the model toward certain compositions and thus generally improves its predictive robustness.


In [ ]:
# Step 1: Get all unique elements
all_elements = sorted(
    {str(el) for s in df_final["Structure"] for el in s.composition.elements}
)
element_to_idx = {el: i for i, el in enumerate(all_elements)}

# Step 2: Build binary matrix (samples × elements)
binary_matrix = np.zeros((len(df_final), len(all_elements)), dtype=int)

for i, s in enumerate(df_final["Structure"]):
    comp = s.composition
    for el in comp:
        binary_matrix[i, element_to_idx[str(el)]] = 1

# Step 3: Stratified multilabel split (preserves element distributions)
sss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, test_idx = next(sss.split(df_final, binary_matrix))

print(f"Train size: {len(train_idx)}, Test size: {len(test_idx)}")

Plot frequency

In [ ]:
def get_element_distribution(indices):
    counter = {el: 0 for el in all_elements}
    for s in df_final.iloc[indices]["Structure"]:
        comp = s.composition
        for el in comp:
            counter[str(el)] += comp[el]
    total = sum(counter.values())
    return {el: counter[el] / total for el in counter}


train_dist = get_element_distribution(train_idx)
test_dist = get_element_distribution(test_idx)

# Plot comparison
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
x = np.arange(len(all_elements))
train_vals = [train_dist[el] for el in all_elements]
test_vals = [test_dist[el] for el in all_elements]

plt.bar(x - 0.2, train_vals, width=0.4, label="Train")
plt.bar(x + 0.2, test_vals, width=0.4, label="Test")
plt.xticks(x, all_elements, rotation=90)
plt.ylabel("Normalized Frequency")
plt.title("Element Frequency Distribution: Train vs Test")
plt.legend()
plt.tight_layout()
plt.show()

### `ElasticSOAPDataset`: On-the-Fly Feature Extraction

The **`ElasticSOAPDataset`** class is a custom **PyTorch** dataset designed to:
1. **Store the raw data** (each row’s `Structure` and corresponding target properties).
2. **Generate SOAP descriptors** for each structure **on the fly**.

Why generate features on the fly?
- **Memory Efficiency**: Storing all SOAP vectors for large datasets can be highly memory-intensive. Creating them on demand avoids holding massive arrays in memory.  
- **Flexibility**: If you wish to alter descriptor parameters (e.g., `r_cut`, `n_max`, `l_max`), the dataset can dynamically re-calculate descriptors without needing to rewrite the entire feature set.  
- **Seamless Integration**: PyTorch’s **`DataLoader`** works seamlessly with datasets that compute features when needed (in each training batch), supporting parallel or background loading if desired.

Overall, this approach keeps the pipeline **modular**, **efficient**, and **easy to maintain**.


In [ ]:
class ElasticSOAPDataset(Dataset):
    def __init__(self, dataframe, soap_params):
        self.df = dataframe
        self.soap = SOAP(**soap_params)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        structure = self.df.loc[idx, "Structure"]

        ase_structure = Atoms(
            symbols=[str(s.specie) for s in structure],
            positions=structure.cart_coords,
            cell=structure.lattice.matrix,
            pbc=True,
        )

        feature = self.soap.create(ase_structure, n_jobs=1).flatten()

        target = self.df.loc[
            idx, ["C11", "C22", "C33", "C44", "C55", "C66"]
        ].values.astype(float)

        return torch.tensor(feature, dtype=torch.float32), torch.tensor(
            target, dtype=torch.float32
        )

In [ ]:
# from sklearn.model_selection import train_test_split

# indices = np.arange(len(df_final))
# train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)


species = list({str(el) for s in df_final["Structure"] for el in s.species})

soap_params = {
    "species": species,
    "periodic": True,
    "r_cut": 6.0,
    "n_max": 5,
    "l_max": 2,
    "average": "inner",
}

train_dataset = ElasticSOAPDataset(
    df_final.iloc[train_idx].reset_index(drop=True), soap_params
)
test_dataset = ElasticSOAPDataset(
    df_final.iloc[test_idx].reset_index(drop=True), soap_params
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Plot Periodic Table with element couting

In [ ]:
import matplotlib.colors as mcolors  # Import colors for LogNorm and explicit colors
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pymatgen.core import Composition, Lattice, Structure
from pymatgen.core.periodic_table import Element, get_el_sp

# 1. Count element frequencies
element_counts = {}
try:
    for s in df_final.iloc[train_idx]["Structure"]:
        comp = s.composition
        for el_sym in comp.keys():  # Use element symbol object from composition
            element = Element(el_sym.symbol)  # Get pymatgen Element from symbol string
            element_counts[element] = element_counts.get(element, 0) + comp[el_sym]
except Exception as e:
    print(f"Error processing structures: {e}. Check df_final format.")
    element_counts = {Element("H"): 1, Element("O"): 2}  # Minimal fallback data

# 2. Normalize frequencies
total_atoms = sum(element_counts.values())
element_freq = {}
if total_atoms > 0:
    # element_freq = {el: count / total_atoms for el, count in element_counts.items()}
    element_freq = {el: count for el, count in element_counts.items()}
else:
    print("Warning: No atoms found in the dataset.")

# --- Plotting Setup ---
MAX_Z = 103  # Define the maximum atomic number to display (e.g., up to Lr)
MISSING_FACE_COLOR = "lightgrey"
MISSING_TEXT_COLOR = "dimgray"
PRESENT_EDGE_COLOR = "gray"

# 3. Prepare Colormap and Normalization for PRESENT elements
cmap = plt.get_cmap("viridis")
norm = None
min_freq = 1.0
max_freq = 0.0

if element_freq:
    present_frequencies = [freq for freq in element_freq.values() if freq > 0]
    if present_frequencies:
        min_freq = min(present_frequencies)
        max_freq = max(present_frequencies)
        # Use LogNorm - ensure vmin is positive and slightly less than actual min
        norm = mcolors.LogNorm(
            vmin=min_freq * 0.9 if min_freq > 0 else 1e-9, vmax=max_freq
        )
    else:
        print("Warning: All element frequencies are zero.")
else:
    print("Warning: No element frequencies calculated.")


# 4. Helper function for element coordinates (includes Ln/Ac placement)
def get_element_display_coords(element: Element):
    """Calculates display row and group for standard periodic table layout."""
    row = element.row
    group = element.group

    if 57 <= element.Z <= 71:  # Lanthanides (La-Lu) -> display row 9
        display_row = 9
        display_group = 3 + (
            element.Z - 57
        )  # Place sequentially starting in group 3 column space
    elif 89 <= element.Z <= 103:  # Actinides (Ac-Lr) -> display row 10
        display_row = 10
        display_group = 3 + (
            element.Z - 89
        )  # Place sequentially starting in group 3 column space
    else:
        display_row = row
        display_group = group

    return display_row, display_group


# 5. Plotting
fig, ax = plt.subplots(figsize=(16, 8))
ax.set_aspect("equal")

# Determine max plot dimensions based on MAX_Z elements' positions
max_disp_row = 0
max_disp_group = 0
all_coords = {}
for Z in range(1, MAX_Z + 1):
    try:
        element = Element.from_Z(Z)
        disp_r, disp_g = get_element_display_coords(element)
        all_coords[Z] = (disp_r, disp_g, element.symbol)
        max_disp_row = max(max_disp_row, disp_r)
        max_disp_group = max(max_disp_group, disp_g)
    except ValueError:
        print(f"Warning: Could not get Element object for Z={Z}")
        continue  # Skip if pymatgen doesn't know the element

ax.set_xlim(0, max_disp_group + 1)
ax.set_ylim(max_disp_row + 1, 0)  # Set limits *before* inverting y-axis
# ax.invert_yaxis()  # Ensure row 1 is at the top


# --- Iterate through ALL elements up to MAX_Z ---
for Z in range(1, MAX_Z + 1):
    if Z not in all_coords:
        continue  # Skip if coordinates weren't determined

    disp_r, disp_g, symbol = all_coords[Z]
    element_obj = Element(symbol)  # Get Element object for checking frequency dict

    face_color = MISSING_FACE_COLOR
    text_color = MISSING_TEXT_COLOR
    edge_color = "silver"  # Lighter edge for missing elements
    label = symbol  # Default label is just the symbol

    # Check if this element is in our frequency data
    if element_obj in element_freq and element_freq[element_obj] > 0 and norm:
        freq_val = element_freq[element_obj]
        face_color = cmap(norm(freq_val))
        edge_color = PRESENT_EDGE_COLOR

        # Determine text color based on background luminance
        luminance = (
            0.2126 * face_color[0] + 0.7152 * face_color[1] + 0.0722 * face_color[2]
        )
        text_color = "white" if luminance < 0.5 else "black"
        # Optional: add frequency to label for present elements
        # label = f"{symbol}\n{freq_val:.2e}" # Show frequency in scientific notation

    # Draw the rectangle (element cell)
    rect = patches.Rectangle(
        (disp_g - 0.5, disp_r - 0.5),
        1,
        1,
        linewidth=0.5,
        edgecolor=edge_color,
        facecolor=face_color,
    )
    ax.add_patch(rect)

    # Add element symbol (or label)
    ax.text(
        disp_g,
        disp_r,
        label,
        ha="center",
        va="center",
        fontsize=9,
        weight="bold" if face_color != MISSING_FACE_COLOR else "normal",
        color=text_color,
    )

# --- Add Group/Row Labels (Optional but Recommended) ---
# Group numbers (columns 1-18)
for i in range(1, max_disp_group + 1):
    # Find the minimum display row for any element in this group (main block or Ln/Ac)
    min_row_in_group = float("inf")
    has_element_in_group = False
    for r, g, _ in all_coords.values():
        if g == i:
            min_row_in_group = min(min_row_in_group, r)
            has_element_in_group = True

    if has_element_in_group:
        # Place label above the highest element in that column
        ax.text(
            i,
            min_row_in_group - 0.8,
            str(i),
            ha="center",
            va="bottom",
            fontsize=9,
            color="gray",
        )

# Row numbers (periods 1-7, plus Ln/Ac labels)
for i in range(1, max_disp_row + 1):
    # Find the minimum display group for any element in this row
    min_group_in_row = float("inf")
    has_element_in_row = False
    label = str(i)  # Default label is row number
    if i == 9:
        label = "Ln"
    if i == 10:
        label = "Ac"

    for r, g, _ in all_coords.values():
        if r == i:
            min_group_in_row = min(min_group_in_row, g)
            has_element_in_row = True

    if has_element_in_row:
        # Place label left of the first element in that row
        ax.text(
            min_group_in_row - 0.8,
            i,
            label,
            ha="right",
            va="center",
            fontsize=9,
            color="gray",
        )


# --- Final Aesthetics ---
# ax.invert_yaxis() # Ensure row 1 is at the top *AFTER* setting limits and plotting
plt.title(f"Element Frequency in Training Set (Z=1-{MAX_Z}, Log Scale)")
plt.axis("off")  # Turn off the outer axis border and ticks

# Add color bar *only* if there were present elements to normalize
if norm:
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Required empty array
    cbar = plt.colorbar(sm, ax=ax, shrink=0.7, aspect=20, pad=0.03)
    cbar.set_label("Relative Element Frequency (Log Scale)")
else:
    # Optional: Add a note if no frequency data is shown
    ax.text(
        0.5,
        -0.05,
        "No frequency data to display.",
        transform=ax.transAxes,
        ha="center",
        fontsize=10,
        color="red",
    )


plt.tight_layout()
plt.show()

## [Step 4]: Model Definition and Training

### Network Architecture

The `ElasticSOAPNet` class defines a simple **fully connected** neural network to predict six diagonal components $\bigl(C_{11}, C_{22}, \ldots, C_{66}\bigr)$. Its key layers are:
- An **input layer** matching the dimensionality of the SOAP descriptor.
- Hidden layers of size 128, 32, and 16, each followed by a **ReLU** activation and **dropout** for regularization.
- An **output layer** producing 6 predicted elastic constants.

```python
class ElasticSOAPNet(nn.Module):
    def __init__(self, input_dim, output_dim=6):
        super(ElasticSOAPNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(16, output_dim),
        )

    def forward(self, x):
        return self.fc(x)
```

### Loss Function

We train the model to minimize the **Mean Squared Error (MSE)** between the predicted ($\hat{\mathbf{y}}_i$) and actual ($\mathbf{y}_i$) elastic tensor components. Formally, for $N$ training samples:

$$
\mathcal{L}(\boldsymbol{\theta}) \;=\; \frac{1}{N} \sum_{i=1}^N \|\mathbf{y}_i - \hat{\mathbf{y}}_i(\boldsymbol{\theta})\|^2,
$$

where $\boldsymbol{\theta}$ represents all learnable parameters in the network.

### Training Procedure

1. **Model Initialization**  
   - Instantiate `ElasticSOAPNet` with the input dimension equal to the SOAP descriptor size and `output_dim=6`.

2. **Optimizer**  
   - We use the **Adam** optimizer with a learning rate of `0.0005` and a small weight decay (`1e-4`) to help regularize the model parameters.

3. **Forward Pass**  
   - For each mini-batch, we compute predicted values $\hat{\mathbf{y}}_i$ by passing the SOAP features through the network.

4. **Compute Loss**  
   - Calculate MSE loss by comparing predictions to true targets for each batch.

5. **Backward Pass**  
   - Perform **backpropagation** to compute gradients with respect to all parameters.

6. **Update Parameters**  
   - The optimizer (Adam) updates each parameter based on the computed gradients.

By iterating through these steps for each epoch, the network progressively **learns** to minimize the MSE loss and provide **accurate predictions** of the target elastic constants.

In [ ]:
import torch.nn as nn


class ElasticSOAPNet(nn.Module):
    def __init__(self, input_dim, output_dim=6):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),  ### <<<<<< TO CANGE
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(16, output_dim),
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
sample_feature, _ = train_dataset[0]
model = ElasticSOAPNet(input_dim=sample_feature.shape[0]).to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [ ]:
from tqdm import tqdm

epochs = 5
model.train()

train_losses = []
val_losses = []

for epoch in range(epochs):
    epoch_loss = 0
    progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit=" batch")

    for i, (features, targets) in enumerate(progress):
        features, targets = features.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        avg_epoch_loss = epoch_loss / (i + 1)

        progress.set_postfix(
            batch_loss=f"{loss.item():.4f}", epoch_avg_loss=f"{avg_epoch_loss:.4f}"
        )

    avg_epoch_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_epoch_loss)

    # Validation loss computation
    model.eval()
    val_epoch_loss = 0
    with torch.no_grad():
        for val_features, val_targets in test_loader:
            val_features, val_targets = val_features.to(device), val_targets.to(device)
            val_outputs = model(val_features)
            val_loss = criterion(val_outputs, val_targets)
            val_epoch_loss += val_loss.item()

    val_epoch_loss /= len(test_loader)
    val_losses.append(val_epoch_loss)

    print(
        f"Epoch [{epoch+1}/{epochs}] Train Loss: {avg_epoch_loss:.4f} | Val Loss: {val_epoch_loss:.4f}"
    )

    model.train()

### Plot the **Learning Curves**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(range(1, epochs + 1), train_losses, label="Training Loss", marker="o")
plt.plot(range(1, epochs + 1), val_losses, label="Validation Loss", marker="o")

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

## [Step 5]: Model Evaluation

After training, we evaluate the performance of our neural network on both the **training** and **test** sets using **MAE** (Mean Absolute Error) and the **$R^2$** coefficient of determination. Here’s how it works:

1. **Gathering Predictions**  
   - We define a helper function `get_predictions(model, loader, device)` which:
     - Sets the model to *evaluation* mode (`model.eval()`).
     - Iterates over the **DataLoader** batches without computing gradients (`torch.no_grad()`).
     - Collects all predicted values and the corresponding ground truth targets.

2. **Metrics**  
   - For each diagonal component $\mathrm{C}_{ii}$:
     - **MAE**: Measures the average absolute difference between predicted and true values.
       $$
         \mathrm{MAE} \;=\; \frac{1}{N}\sum_{i=1}^N \bigl| y_i - \hat{y}_i \bigr|
       $$
     - **$R^2$**: A measure of how much variance in the target is explained by the model, with 1.0 being perfect:
       $$
         R^2 \;=\; 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}.
       $$
       Here $\bar{y}$ is the mean of the true values.

3. **Parity Plots**  
   - We create a **scatter plot** of **Actual vs. Predicted** for each component. 
   - An **identity line** ($y=x$) helps visualize how close predictions are to the ground truth.
   - We overlay **train** and **test** points in different colors, making it easy to compare **overfitting** (train points hugging the line but test points diverging) or good **generalization** (both sets closely match).

These visual and numerical evaluations provide a clear picture of the model’s **accuracy** and **robustness** in predicting the six diagonal components of the elastic tensor.


In [ ]:
def get_predictions(model, loader, device):
    model.eval()
    preds, actuals = [], []
    with torch.no_grad():
        for features, targets in tqdm(loader, desc="Evaluating"):
            features = features.to(device)
            outputs = model(features).cpu().numpy()
            preds.append(outputs)
            actuals.append(targets.numpy())
    return np.vstack(actuals), np.vstack(preds)


# Predictions on Train and Test datasets
train_actuals, train_preds = get_predictions(model, train_loader, device)
test_actuals, test_preds = get_predictions(model, test_loader, device)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score

tensor_components = ["C11", "C22", "C33", "C44", "C55", "C66"]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle("Parity Plots: Train vs Test Overlay", fontsize=18)

for i, comp in enumerate(tensor_components):
    # Train metrics
    train_mae = mean_absolute_error(train_actuals[:, i], train_preds[:, i])
    train_r2 = r2_score(train_actuals[:, i], train_preds[:, i])

    # Test metrics
    test_mae = mean_absolute_error(test_actuals[:, i], test_preds[:, i])
    test_r2 = r2_score(test_actuals[:, i], test_preds[:, i])

    ax = axes.reshape(-1)[i]
    ax.scatter(
        train_actuals[:, i], train_preds[:, i], alpha=0.6, label="Train", color="blue"
    )
    ax.scatter(
        test_actuals[:, i], test_preds[:, i], alpha=0.6, label="Test", color="orange"
    )

    # Identity line
    all_vals = np.concatenate(
        [train_actuals[:, i], train_preds[:, i], test_actuals[:, i], test_preds[:, i]]
    )
    min_val, max_val = all_vals.min(), all_vals.max()
    ax.plot([min_val, max_val], [min_val, max_val], "r--")

    ax.set_title(
        f"{comp}\nTrain MAE: {train_mae:.1f}, R²: {train_r2:.2f}\nTest MAE: {test_mae:.1f}, R²: {test_r2:.2f}"
    )
    ax.set_xlabel("Actual (GPa)")
    ax.set_ylabel("Predicted (GPa)")
    ax.grid(True)
    ax.legend()

plt.tight_layout(rect=[0, 0.05, 1, 0.9])
plt.show()

# Verification (Our model VS MACE)

### Downloading and Uploading a Crystal Structure from Materials Project

1. **Go to Materials Project**  
   - Visit [materialsproject.org](https://materialsproject.org/).  
   - If you are not already signed in, create a free account or log in.  

2. **Search for "Silicon"**  
   - In the search bar, type "Silicon" or use the chemical formula "Si."  
   - You’ll see one or more entries for stable/low-energy structures (e.g., mp-149).  

3. **Download the CIF**  
   - Open the structure entry you are interested in.  
   - Click on the **"Download"** or **"Export"** button and select **CIF** to save the file locally (e.g., `Si.cif`).  

4. **Upload to Google Colab**  
   - In your Colab notebook interface, look on the left sidebar for the **Files** tab.  
   - Drag and drop the downloaded `Si.cif` file into the **home folder** (or any target folder where your code can access it).  

5. **Repeat for Your Assigned Structure**  
   - For the assignment at home, follow the same steps for the structure given in your **group assignment**.  
   - Download its CIF from Materials Project or a similar database.  
   - Upload it into Colab (e.g., `MyStructure.cif`) so your notebook can read it.

Once uploaded, you can confirm the file is there by listing the directory contents inside Colab, for example:
```python
!ls
```

This ensures you have the crystal structure files ready for **featurization** (SOAP, etc.) and subsequent **elastic constant predictions**.
```axes

In [ ]:
import numpy as np
import torch
from ase.io import read
from dscribe.descriptors import SOAP

FILE_NAME = "Si.cif"

# === Load the structure ===
atoms = read("Si.cif")

# === Recreate the SOAP descriptor ===
# Assumes soap_params is already defined as before
soap = SOAP(**soap_params)

# === Create descriptor for the structure ===
feature = soap.create(atoms, n_jobs=-1).flatten()
feature_tensor = (
    torch.tensor(feature, dtype=torch.float32).unsqueeze(0).to(device)
)  # batch size = 1
del feature

# === Load the trained model ===
# model.eval()

# === Predict elastic properties ===
with torch.no_grad():
    prediction = model(feature_tensor).cpu().numpy().flatten()

# === Output ===
tensor_components = ["C11", "C22", "C33", "C44", "C55", "C66"]
print("Predicted elastic tensor components (GPa):")
for comp, value in zip(tensor_components, prediction):
    print(f"{comp}: {value:.2f}")
del feature_tensor

## Using MACE to Compute the Stress Tensor and Extract Elastic Constants

### Theoretical Background

For a **small strain** $\boldsymbol{\varepsilon}$, the change in total energy $\Delta E$ relative to the unstrained reference ($\varepsilon = 0$) can be approximated by:

$$
\Delta E(\boldsymbol{\varepsilon}) \;=\; \frac{1}{2}\, V \, \boldsymbol{\varepsilon}^T \, \mathbf{C} \, \boldsymbol{\varepsilon},
$$

where:
- $\mathbf{C}$ is the **elastic stiffness matrix** (or specific components of it),
- $V$ is the **equilibrium volume** of the crystal,
- $\boldsymbol{\varepsilon}$ is the strain vector (in Voigt notation for simplicity).

Focusing on a single elastic constant $C_{ij}$, we often apply **one** independent component of strain $ \varepsilon $ at a time to the reference cell. Then the energy change simplifies to:

$$
\Delta E(\varepsilon) \;\approx\; \tfrac{1}{2}\, C_{ij}\, V\, \varepsilon^2.
$$

By **polynomial-fitting** $\Delta E$ vs. $\varepsilon^2$ over small strains (e.g., $\varepsilon \in [-0.01,\,+0.01]$), we solve for $C_{ij}$. 

### Using MACE in Practice

[MACE](https://github.com/ACEsuit/mace) is a **machine-learned interatomic potential** trained on quantum-mechanical data. It can compute **forces**, **stresses**, and **energies** for a given atomic configuration.

1. **Attach the MACE Calculator**  
   ```python
   calculator = MACECalculator(
       model_path="MACE-matpes-r2scan-omat-ft.model",
       device=device.type
   )
   atoms_ref.calc = calculator
   ```
   This tells the `ase.Atoms` object to use MACE for its energy and force evaluations.

2. **Apply Small Strains**  
   - We define a helper function `apply_strain(atoms, strain_tensor)` that modifies the cell dimensions according to $\boldsymbol{\varepsilon}$.  
   - A loop is used to vary $\varepsilon$ from negative to positive (e.g., $-1\%$ to $+1\%$).

3. **Compute the Energy**  
   - For each strained geometry, MACE calculates the total potential energy:
     ```python
     energy = atoms_def.get_potential_energy()
     ```
   - We record these energies in an array, subtract the reference (unstrained) energy to get $\Delta E(\varepsilon)$.

4. **Extract $ C_{ij} $**  
   - Fit $\Delta E(\varepsilon)$ vs. $\varepsilon^2$ using a simple linear regression (or polynomial fit) to solve:
     $$
       \Delta E(\varepsilon) \;=\; a\, \varepsilon^2 + b\,\varepsilon + c
     $$
     Typically, $b$ and $c$ should be near zero for purely elastic (symmetric) strains, and $a = \tfrac{1}{2}\,C_{ij}\,V$.  
   - Rearrange to find $C_{ij}$:
     $$
       C_{ij} \;=\; \frac{2\,a}{V}.
     $$

5. **Interpretation**  
   - Repeating for different strain “modes” (e.g., $\varepsilon_{11}, \varepsilon_{22}, \ldots$, or shear strains) lets us extract various diagonal (and sometimes off-diagonal) **elastic constants**.  
   - These computed values can be **compared** directly to the model predictions (from the neural network) or **literature** data.

This approach leverages MACE’s **ab initio–level accuracy** (learned from high-fidelity quantum-mechanical calculations) but at significantly lower computational cost than running a full DFT simulation for each strain configuration. The result is a **fast and flexible** way to obtain elastic constants by **direct energy or stress evaluations**.


In [ ]:
!wget https://github.com/ACEsuit/mace-foundations/releases/download/mace_matpes_0/MACE-matpes-r2scan-omat-ft.model

In [ ]:
import copy

import matplotlib.pyplot as plt
import numpy as np
from ase import Atoms
from ase.io import read
from ase.units import GPa
from mace.calculators import MACECalculator

In [ ]:
def apply_strain(atoms: Atoms, strain_tensor):
    atoms = atoms.copy()
    cell = atoms.get_cell()
    strain_matrix = np.eye(3) + strain_tensor
    atoms.set_cell(np.dot(strain_matrix, cell), scale_atoms=True)
    return atoms


def compute_elastic_constant(strain_energies, strains, volume):
    # Fit E = 0.5 * C * strain^2 → C = 2 * dE/d(strain^2)
    coeffs = np.polyfit(strains**2, strain_energies, 1)
    return (2 * coeffs[0]) / volume / GPa  # in GPa

In [ ]:
# Load structure
atoms_ref = read("Si.cif")
volume = atoms_ref.get_volume()

# Attach MACE-MP0 calculator
calculator = MACECalculator(
    model_path="MACE-matpes-r2scan-omat-ft.model", device=device.type
)
atoms_ref.calc = calculator

# Small strains to apply
strain_values = np.linspace(-0.01, 0.01, 11)

# Tensor components to compute
component_labels = ["C11", "C22", "C33", "C44", "C55", "C66"]
strain_tensors = {
    "C11": lambda e: np.array([[e, 0, 0], [0, 0, 0], [0, 0, 0]]),
    "C22": lambda e: np.array([[0, 0, 0], [0, e, 0], [0, 0, 0]]),
    "C33": lambda e: np.array([[0, 0, 0], [0, 0, 0], [0, 0, e]]),
    "C44": lambda e: np.array([[0, e / 2, 0], [e / 2, 0, 0], [0, 0, 0]]),
    "C55": lambda e: np.array([[0, 0, e / 2], [0, 0, 0], [e / 2, 0, 0]]),
    "C66": lambda e: np.array([[0, 0, 0], [0, 0, e / 2], [0, e / 2, 0]]),
}

elastic_constants = {}
all_atoms_def = [atoms_ref.copy()]
for label in component_labels:
    energies = []
    for strain in strain_values:
        atoms_def = apply_strain(atoms_ref, strain_tensors[label](strain))
        atoms_def.calc = calculator
        energy = atoms_def.get_potential_energy()
        energies.append(energy)
        all_atoms_def.append(atoms_def.copy())
    energies = np.array(energies)
    strains = np.array(strain_values)
    dE = energies - energies[strains == 0][0]
    Cij = compute_elastic_constant(dE, strains, volume)
    plt.plot(dE, strains)
    plt.ylabel(r"$\Delta E$ (eV)")
    plt.xlabel(rf"$\epsilon_{{{label[1:]}}}$ (-)")
    plt.show()
    elastic_constants[label] = Cij
    print(f"{label}: {Cij:.2f} GPa")

In [ ]:
for label, Cij in elastic_constants:
    print(f"{label}: {Cij:.2f} GPa")

In [ ]:
from ase.visualize import view

view(all_atoms_def, viewer="ngl")